In [1]:
import numpy as np

import dolfinx
import ufl
from dolfinx.fem import Function, FunctionSpace, dirichletbc, Constant, Expression
from dolfinx.fem.petsc import LinearProblem, NonlinearProblem
from dolfinx.mesh import CellType, create_rectangle
from ufl import (FiniteElement, MixedElement, VectorElement, dx, grad, inner,
                 split)
import pyvista
from dolfinx import plot

from mpi4py import MPI
from petsc4py import PETSc


## Parameters of the geometry and the material

In [2]:
# the radius of the cylinder
r = 1.016 

# the length of the cylinder
L = 3.048 

# the Young's modulus and Poisson's ratio
E, nu = 2.0685E7, 0.3 

# the shear modulus
mu = E/(2.0*(1.0 + nu)) 

# the lame parameter
lmbda = 2.0*mu*nu/(1.0 - 2.0*nu) 

# the thickness of the cylinder
t = 0.03 

# Generate the mesh in the parameter space 
$x_1 \in [- \pi / 2, \pi /2], x_2 \in [0, L]$

In [3]:
mesh = create_rectangle(MPI.COMM_WORLD, np.array([[-np.pi / 2, 0], [np.pi / 2, L]]), [20, 20], CellType.triangle)
tdim = mesh.topology.dim

## Plot the parameter space

In [5]:
pyvista.start_xvfb()

topology, cell_types, geometry = plot.vtk_mesh(mesh, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()

plotter.show()

Widget(value='<iframe src="http://localhost:43253/index.html?ui=P_0x7fd95ba6ebd0_1&reconnect=auto" class="pyvi…

# Define the initial shape

In [4]:
x = ufl.SpatialCoordinate(mesh)
phi0_ufl = ufl.as_vector([r * ufl.sin(x[0]), x[1], r * ufl.cos(x[0])])

In [ ]:

phi_FS = FunctionSpace(mesh, VectorElement("Lagrange", ufl.triangle, degree = 2, dim = 3))
phi0_expr = Expression(phi0_ufl, phi_FS.element.interpolation_points())

phi0_func = Function(phi_FS)
phi0_func.interpolate(phi0_expr)


## Plot the initial shape

In [7]:
topology, cell_types, x_para_space = plot.vtk_mesh(phi_FS)

x_init_shape = phi0_func.x.array.reshape((x_para_space.shape[0], len(phi0_func)))

grid_ps = pyvista.UnstructuredGrid(topology, cell_types, x_para_space)
grid_is = pyvista.UnstructuredGrid(topology, cell_types, x_init_shape)

plotter = pyvista.Plotter()
plotter.add_mesh(grid_ps, style="wireframe", color="k")
plotter.add_mesh(grid_is, show_edges=True)

plotter.show()



Widget(value='<iframe src="http://localhost:43253/index.html?ui=P_0x7fd89c251e50_2&reconnect=auto" class="pyvi…

# Define the initial local orthonormal basis


## Unit normal basis 
$$
 \vec{n}  = \frac{\partial_1 \phi_0 \times \partial_2 \phi_0}{\| \partial_1 \phi_0 \times \partial_2 \phi_0 \|}
$$

In [5]:
def unit_normal(phi):
    n = ufl.cross(phi.dx(0), phi.dx(1))
    return n/ufl.sqrt(inner(n, n))

# the analytical expression of n0
n0_ufl = unit_normal(phi0_ufl)


### Plot the normal vectors on the initial shape

In [9]:

# create a FEM interpolation of n0 
n_FS = FunctionSpace(mesh, VectorElement("Lagrange", ufl.triangle, degree = 1, dim = 3))

n0_expr = Expression(n0_ufl, n_FS.element.interpolation_points())
n0_func = Function(n_FS)
n0_func.interpolate(n0_expr)

In [10]:
phi0_P1_expr = Expression(phi0_ufl, n_FS.element.interpolation_points())

phi0_P1 = Function(n_FS)
phi0_P1.interpolate(phi0_P1_expr)


#pyvista.start_xvfb()
topology, cell_types, geometry = plot.vtk_mesh(n_FS)


geometry_phi0_P1 = phi0_P1.x.array.reshape((geometry.shape[0], len(phi0_P1)))
geometry_n0 = n0_func.x.array.reshape((geometry.shape[0], len(n0_func)))

grid_phi0_P1 = pyvista.UnstructuredGrid(topology, cell_types, geometry_phi0_P1)
grid_phi0_P1["n0"] = geometry_n0
glyphs = grid_phi0_P1.glyph(orient="n0", factor=0.2)


plotter = pyvista.Plotter()
plotter.add_mesh(grid_phi0_P1, style="wireframe", color="k")
plotter.add_mesh(glyphs,  show_scalar_bar=True)

plotter.show()

Widget(value='<iframe src="http://localhost:43253/index.html?ui=P_0x7fda041b1ac0_3&reconnect=auto" class="pyvi…

## two tangent basis
$$
\vec{t}_{0i} = \mathbf{R}_0 \vec{e}_i \\
\vec{n} = \vec{t}_{03} \\
$$

Define $\vec{t}_{01}$ and $\vec{t}_{02}$ with $\vec{e}_1$ and $\vec{e}_2$ 
$$
\vec{t}_{01} = \frac{\vec{e}_2 \times \vec{n}}{\| \vec{e}_2 \times \vec{n}\|} \\
\vec{t}_{02} =   \vec{n} \times \vec{t}_{01}
$$

The corresponding rotation matrix $\mathbf{R}_0$:
$$
\mathbf{R}_0 = [\vec{t}_{01}; \vec{t}_{02}; \vec{n}]
$$



In [6]:
def tangent_1(n):
    e2 = ufl.as_vector([0, 1, 0])
    t1 = ufl.cross(e2, n)
    t1 = t1/ufl.sqrt(inner(t1, t1))
    return t1

def tangent_2(n, t1):
    t2 = ufl.cross(n, t1)
    t2 = t2/ufl.sqrt(inner(t2, t2))
    return t2

# the analytical expression of t1 and t2
t1_ufl = tangent_1(n0_ufl)
t2_ufl = tangent_2(n0_ufl, t1_ufl)

# the analytical expression of R0
def rotation_matrix(t1, t2, n):
    R = ufl.as_matrix([[t1[0], t2[0], n[0]], 
                       [t1[1], t2[1], n[1]], 
                       [t1[2], t2[2], n[2]]])
    return R

R0_ufl = rotation_matrix(t1_ufl, t2_ufl, n0_ufl)

### plot the tangent basis

In [12]:
# create a FEM interpolation of t1, t2, n0 and phi0
t_FS = FunctionSpace(mesh, VectorElement("Lagrange", ufl.triangle, degree = 1, dim = 3))

t1_expr = Expression(t1_ufl, t_FS.element.interpolation_points())
t1_func = Function(t_FS)
t1_func.interpolate(t1_expr)

t2_expr = Expression(t2_ufl, t_FS.element.interpolation_points())
t2_func = Function(t_FS)
t2_func.interpolate(t2_expr)

n0_expr = Expression(n0_ufl, t_FS.element.interpolation_points())
n0_func = Function(t_FS)
n0_func.interpolate(n0_expr)

phi0_P1_expr = Expression(phi0_ufl, t_FS.element.interpolation_points())
phi0_P1 = Function(t_FS)
phi0_P1.interpolate(phi0_P1_expr)

In [13]:
topology, cell_types, geometry = plot.vtk_mesh(t_FS)

geometry_phi0_P1 = phi0_P1.x.array.reshape((geometry.shape[0], len(phi0_P1)))
geometry_t1 = t1_func.x.array.reshape((geometry.shape[0], len(t1_func)))
geometry_t2 = t2_func.x.array.reshape((geometry.shape[0], len(t2_func)))
geometry_n0 = n0_func.x.array.reshape((geometry.shape[0], len(n0_func)))

grid_phi0_P1 = pyvista.UnstructuredGrid(topology, cell_types, geometry_phi0_P1)

grid_phi0_P1["n0"] = geometry_n0
grid_phi0_P1["t1"] = geometry_t1
grid_phi0_P1["t2"] = geometry_t2

glyphs_n0 = grid_phi0_P1.glyph(orient="n0", factor=0.1)
glyphs_t1 = grid_phi0_P1.glyph(orient="t1", factor=0.1)
glyphs_t2 = grid_phi0_P1.glyph(orient="t2", factor=0.1)

plotter = pyvista.Plotter()
plotter.add_mesh(grid_phi0_P1, style="wireframe", color="k", line_width= 0.5)
plotter.add_mesh(glyphs_n0, color = 'b', show_scalar_bar=True)
plotter.add_mesh(glyphs_t1, color = 'r', show_scalar_bar=True)
plotter.add_mesh(glyphs_t2, color = 'g', show_scalar_bar=True)
plotter.camera_position = 'xz'
plotter.camera.azimuth = -45
plotter.camera.elevation = 15
plotter.show_axes_all()

plotter.show()

Widget(value='<iframe src="http://localhost:43253/index.html?ui=P_0x7fd89c16dcd0_4&reconnect=auto" class="pyvi…

## Test the rotation matrix

In [14]:
e1 = ufl.as_vector([1, 0, 0])
e2 = ufl.as_vector([0, 1, 0])
e3 = ufl.as_vector([0, 0, 1])

t1_test_ufl = ufl.dot(R0_ufl, e1)
t2_test_ufl = ufl.dot(R0_ufl, e2)
n0_test_ufl = ufl.dot(R0_ufl, e3)

### Plot the test vectors

In [15]:
# create a FEM interpolation of t1, t2, n0 and phi0
t_FS = FunctionSpace(mesh, VectorElement("Lagrange", ufl.triangle, degree = 1, dim = 3))

t1_test_expr = Expression(t1_test_ufl, t_FS.element.interpolation_points())
t1_test_func = Function(t_FS)
t1_test_func.interpolate(t1_test_expr)

t2_test_expr = Expression(t2_test_ufl, t_FS.element.interpolation_points())
t2_test_func = Function(t_FS)
t2_test_func.interpolate(t2_test_expr)

n0_test_expr = Expression(n0_test_ufl, t_FS.element.interpolation_points())
n0_test_func = Function(t_FS)
n0_test_func.interpolate(n0_test_expr)

phi0_P1_expr = Expression(phi0_ufl, t_FS.element.interpolation_points())
phi0_P1 = Function(t_FS)
phi0_P1.interpolate(phi0_P1_expr)

In [16]:
topology, cell_types, geometry = plot.vtk_mesh(t_FS)

geometry_phi0_P1 = phi0_P1.x.array.reshape((geometry.shape[0], len(phi0_P1)))

geometry_t1_test = t1_test_func.x.array.reshape((geometry.shape[0], len(t1_test_func)))
geometry_t2_test = t2_test_func.x.array.reshape((geometry.shape[0], len(t2_test_func)))
geometry_n0_test = n0_test_func.x.array.reshape((geometry.shape[0], len(n0_test_func)))

grid_phi0_P1 = pyvista.UnstructuredGrid(topology, cell_types, geometry_phi0_P1)

grid_phi0_P1["n0"] = geometry_n0_test
grid_phi0_P1["t1"] = geometry_t1_test
grid_phi0_P1["t2"] = geometry_t2_test

glyphs_n0 = grid_phi0_P1.glyph(orient="n0", factor=0.1)
glyphs_t1 = grid_phi0_P1.glyph(orient="t1", factor=0.1)
glyphs_t2 = grid_phi0_P1.glyph(orient="t2", factor=0.1)

plotter = pyvista.Plotter()
plotter.add_mesh(grid_phi0_P1, style="wireframe", color="k", line_width= 0.5)
plotter.add_mesh(glyphs_n0, color = 'b', show_scalar_bar=True)
plotter.add_mesh(glyphs_t1, color = 'r', show_scalar_bar=True)
plotter.add_mesh(glyphs_t2, color = 'g', show_scalar_bar=True)
plotter.camera_position = 'xz'
plotter.camera.azimuth = -45
plotter.camera.elevation = 15
plotter.show_axes_all()

plotter.show()

Widget(value='<iframe src="http://localhost:43253/index.html?ui=P_0x7fd89c160e30_5&reconnect=auto" class="pyvi…

# The parameterization of director 
Update the director with two successive elementary rotations
$$
    \vec{t}_3 = \mathbf{R}_0 \cdot \vec{\Lambda_3}
$$

$$
\vec{\Lambda_3} = [\sin(\theta_2)\cos(\theta_1), -\sin(\theta_1), \cos(\theta_2)\cos(\theta_1)]^\text{T}
$$

Derivation: 


$\theta_1$, $\theta_2$ are the rotation angles about the fixed axis $\vec{e}_1$ and $\vec{e}_2$ or follower axes $\mathbf{t}_1$ and $\mathbf{t}_2$

$$
\vec{t}_i = \mathbf{R} \vec{e}_i \\

\mathbf{R}  = \text{exp}[\theta_1 \hat{\mathbf{t}}_1] \text{exp}[\theta_2 \hat{\mathbf{t}}_{02}] \mathbf{R}_0 
$$

where $\mathbf{t}_{02} = \mathbf{R}_0 \cdot \vec{e}_2 $, $\mathbf{t}_1 = \text{exp}[\theta_2 \hat{\mathbf{t}}_{02}] \cdot \mathbf{t}_{01} $

$$
\mathbf{R} = \mathbf{R}_0 \text{exp}[\theta_2 \hat{\mathbf{e}}_{2}] \text{exp}[\theta_1 \hat{\mathbf{e}}_1]

$$

In [7]:
# Update the director with two successive elementary rotations

def director(R0, theta):
    Lm3 = ufl.as_vector([ufl.sin(theta[1])*ufl.cos(theta[0]), -ufl.sin(theta[0]), ufl.cos(theta[1])*ufl.cos(theta[0])])
    d = ufl.dot(R0, Lm3)
    return d

## Test the director 

In [28]:
theta_ufl = ufl.as_vector([np.pi/4, np.pi/2])

theta_const = Constant(mesh, [np.pi/4, np.pi/2])

theta_test_FS = FunctionSpace(mesh, VectorElement("Lagrange", ufl.triangle, degree=2, dim=2))
theta_test_expr = Expression(theta_ufl, theta_test_FS.element.interpolation_points(), comm=mesh.comm)
theta_test_func = Function(theta_test_FS)
theta_test_func.interpolate(theta_test_expr)

# d0_ufl = director(R0_ufl, theta_ufl)
# d0_ufl = director(R0_ufl, theta_const)
d0_ufl = director(R0_ufl, theta_test_func)

d_FS = FunctionSpace(mesh, VectorElement("Lagrange", ufl.triangle, degree = 1, dim = 3))

d0_expr = Expression(d0_ufl, d_FS.element.interpolation_points())
d0_func = Function(d_FS)
d0_func.interpolate(d0_expr)

phi0_P1_expr = Expression(phi0_ufl, d_FS.element.interpolation_points())
phi0_P1 = Function(d_FS)
phi0_P1.interpolate(phi0_P1_expr)


In [29]:
topology, cell_types, geometry = plot.vtk_mesh(d_FS)

geometry_phi0_P1 = phi0_P1.x.array.reshape((geometry.shape[0], len(phi0_P1)))

geometry_d0 = d0_func.x.array.reshape((geometry.shape[0], len(d0_func)))

grid_phi0_P1 = pyvista.UnstructuredGrid(topology, cell_types, geometry_phi0_P1)

grid_phi0_P1["d0"] = geometry_d0


glyphs_d0 = grid_phi0_P1.glyph(orient="d0", factor=0.1)

plotter = pyvista.Plotter()
plotter.add_mesh(grid_phi0_P1, style="wireframe", color="k", line_width= 0.5)
plotter.add_mesh(glyphs_d0)

# plotter.camera_position = 'xz'
# plotter.camera.azimuth = -45
# plotter.camera.elevation = 15

plotter.enable_parallel_projection()
plotter.view_xz()
plotter.show_axes_all()

plotter.show()

Widget(value='<iframe src="http://localhost:43253/index.html?ui=P_0x7fd87c114e30_8&reconnect=auto" class="pyvi…

# Nonlinear Naghdi Shell elements

## Define element

For 3 translations $u_x$, $u_y$, and $u_z$, we use 2nd order Langrange elements enriched with 3rd order Bubble elements

For 2 rotations $\theta_1$ and $\theta_2$, we use 2nd order Langrange elements.

In [8]:
# for the 3 translation DOFs, we use the P2 + B3 enriched element
P2_d3 = VectorElement("Lagrange", ufl.triangle, degree = 2, dim = 3)
B3_d3 = VectorElement("Bubble", ufl.triangle, degree = 3, dim = 3)

P2B3_d3 = P2_d3 + B3_d3

# for 2 rotation DOFs, we use P2 element
P2_d2 = VectorElement("Lagrange", ufl.triangle, degree=2, dim=2)

# mixed element for the nonlinear Naghdi shell
naghdi_shell_element = MixedElement(P2B3_d3, P2_d2)
naghdi_shell_FS = FunctionSpace(mesh, naghdi_shell_element)

## Define `Function`

In [9]:
q_func = Function(naghdi_shell_FS)
q_trial = ufl.TrialFunction(naghdi_shell_FS)
q_test = ufl.TestFunction(naghdi_shell_FS)

u_func, theta_func = split(q_func)

## Define metric and curvature tensor

Deformation gradient
$$
\mathbf{F} = \nabla \vec{\phi} \quad  (F_{ij} = \frac{\partial \phi_i}{\partial \xi_j}); \quad \vec{\phi} = \vec{\phi}_0 + 
\vec{u}
$$

Metric tensor $\mathbf{a} \in \mathbb{S}^2_+$ and curvature tensor $\mathbf{b} \in \mathbb{S}^2$ (First and second fundamental form)
$$
\mathbf{a} = {\nabla \vec{\phi}} ^{T} \nabla \vec{\phi} \\
\mathbf{b} = -\frac{1}{2}({\nabla \vec{\phi}} ^{T} \nabla \vec{d} + {\nabla \vec{d}} ^{T} \nabla \vec{\phi})

$$

Initial configuration, $\vec{d} = \vec{n}_0$, $\vec{\phi} = \vec{\phi}_0$, conresponding initial tensors $\mathbf{a}_0$, $\mathbf{b}_0$

In [25]:
# current deformation gradient 
F = grad(u_func) + grad(phi0_ufl) 

# current director
d = director(R0_ufl, theta_func)

# initial metric and curvature tensor a0 and b0
a0_ufl = grad(phi0_ufl).T * grad(phi0_ufl)
b0_ufl = -0.5*( grad(phi0_ufl).T * grad(n0_ufl) + grad(n0_ufl).T * grad(phi0_ufl) )

## Define strain measures

- Membrane strain tensor $\boldsymbol{\varepsilon}(\vec{u})$

$$
\boldsymbol{\varepsilon} (\vec{u})= \frac{1}{2} \left ( \mathbf{a}(\vec{u}) - \mathbf{a}_0 \right)
$$

- Bending strain tensor $\boldsymbol{\kappa}(\vec{u}, \vec{\theta})$ 

$$
\boldsymbol{\kappa}(\vec{u}, \vec{\theta}) = \mathbf{b}(\vec{u}, \vec{\theta}) - \mathbf{b}_0
$$

- transverse shear strain vector $\vec{\gamma}(\vec{u}, \vec{\theta})$ 

$$
\begin{aligned}
\vec{\gamma}(\vec{u}, \vec{\theta}) & = {\nabla \vec{\phi}(\vec{u})}^T \vec{d}(\vec{\theta}) - {\nabla\vec{\phi}_0}^T \vec{n}_0 \\
& = {\nabla \vec{\phi}(\vec{u})}^T \vec{d}(\vec{\theta}) \quad \text{if zero initial shears}
\end{aligned}
$$



In [26]:
# membrane strain
epsilon = lambda F: 0.5*(F.T * F - a0_ufl)

# bending strain
kappa = lambda F, d: -0.5 * (F.T * grad(d) + grad(d).T * F) - b0_ufl

# transverse shear strain (zero initial shear strain)
gamma = lambda F, d: F.T * d

## Define isotropic linear material model

In [27]:
a0_contra_ufl = ufl.inv(a0_ufl)
j0_ufl = ufl.det(a0_ufl)

i,j,l,m = ufl.indices(4)
A_contra_ufl = ufl.as_tensor( ( ((2.0*lmbda*mu) / (lmbda + 2.0*mu)) * a0_contra_ufl[i,j]*a0_contra_ufl[l,m]
                + 1.0*mu* (a0_contra_ufl[i,l]*a0_contra_ufl[j,m] + a0_contra_ufl[i,m]*a0_contra_ufl[j,l]) )
                ,[i,j,l,m])


## Define stress measures

- Membrane stress tensor $\mathbf{N}$
- Bending stress tensor $\mathbf{M}$
- Shear stress vector $\vec{T}$


In [28]:
N = ufl.as_tensor(t * A_contra_ufl[i,j,l,m] * epsilon(F)[l,m], [i,j])

M = ufl.as_tensor( (t**3 / 12.0) * A_contra_ufl[i,j,l,m]*kappa(F, d)[l,m], [i,j])

T = ufl.as_tensor( (t * mu) * a0_contra_ufl[i, j] * gamma(F, d)[j], [i])

## Define elastic strain energy density
$\psi_{m}$, $\psi_{b}$, $\psi_{s}$ for membrane, bending and shear, respectively.

$$
\psi_m = \frac{1}{2} \mathbf{N} : \boldsymbol{\varepsilon}; \quad
\psi_b = \frac{1}{2} \mathbf{M} : \boldsymbol{\kappa}; \quad
\psi_s = \frac{1}{2} \vec{T} \cdot \vec{\gamma}
$$

In [29]:
psi_m = 0.5*inner(N, epsilon(F))

psi_b = 0.5*inner(M, kappa(F, d))

psi_s = 0.5*inner(T, gamma(F, d))